In [ ]:
import cooklang
import pandas as pd
import os

import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import (
    BootstrapFewShotWithRandomSearch,
    MIPROv2,
)
import json

from programs import (
    CookLangFormatter,
    CookLangFormatterNoSteps,
    CookLangFormatterNoStepsNoIngredients,
    CookLangSignature,
)
from dotenv import load_dotenv

In [ ]:
# load your environment variables from .env file
load_dotenv()

# azure-openai model deployment
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_4o_mini = os.getenv("AZURE_OPENAI_DEPLOYMENT_4o_mini")
AZURE_OPENAI_DEPLOYMENT_4o = os.getenv("AZURE_OPENAI_DEPLOYMENT_4o")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

# ollama deployment
OLLAMA_URL = os.getenv("OLLAMA_URL")

In [ ]:
def parse_ingredients(recipe: str):
    cooklang_recipe = cooklang.parseRecipe(recipe.replace("--", " "))
    ingredients = cooklang_recipe["ingredients"]

    ingredients = [
        f"{i['quantity']} {i['units']} {i['name']}".strip().replace(".000", "")
        for i in ingredients
    ]
    return ",".join(ingredients)


df = pd.read_csv("data/bronze/cooklang_dataset_with_method.tsv", sep="\t")
df["ingredients"] = df["cooklang"].apply(parse_ingredients)
df["ingredients"] = df["ingredients"].fillna("No ingredients.")

df.head()

In [ ]:
def create_example(row: pd.Series) -> dspy.Example:
    return dspy.Example(
        ingredients=row["ingredients"],
        recipe_text=row["method"],
        cooklang=row["cooklang"],
    ).with_inputs("ingredients", "recipe_text")


def create_example_no_ingredient(row: pd.Series) -> dspy.Example:
    return dspy.Example(
        recipe_text=row["method"],
        cooklang=row["cooklang"],
    ).with_inputs("recipe_text")


examples = []
for _, row in df.iterrows():
    example = create_example(row)
    examples.append(example)

examples_no_ingredients = []
for _, row in df.iterrows():
    example = create_example_no_ingredient(row)
    examples_no_ingredients.append(example)

In [ ]:
def get_ingredients(recipe: str):
    cooklang_recipe = cooklang.parseRecipe(recipe.replace("--", " "))
    ingredients = cooklang_recipe["ingredients"]
    return [i for i in ingredients]


def compare_arrays(true_arr, pred_arr):
    true_set = set(json.dumps(d, sort_keys=True) for d in true_arr)
    pred_set = set(json.dumps(d, sort_keys=True) for d in pred_arr)

    all_true_in_pred = true_set.issubset(pred_set)

    # Find bonus pred values
    bonus_pred = [json.loads(item) for item in pred_set - true_set]

    return all_true_in_pred, bonus_pred


def validate_answer(
    example: dspy.Example, pred: CookLangSignature, trace: object = None
) -> bool:
    try:
        true_cooklang = example.cooklang
        pred_cooklang = pred.cooklang

        true_ingredients = get_ingredients(true_cooklang)
        pred_ingredients = get_ingredients(pred_cooklang)

        # Run the comparison
        all_true_in_pred, bonus_pred = compare_arrays(
            true_ingredients, pred_ingredients
        )

        if all_true_in_pred:
            score = True
        else:
            score = False
    except Exception:
        score = False
    return score


evaluate = Evaluate(
    devset=examples,
    metric=validate_answer,
    num_threads=1,
    display_progress=True,
    display_table=0,
)

# Programs

In [ ]:
models = [
    "llama3.1:8b",
    "llama3.1:70b",
    AZURE_OPENAI_DEPLOYMENT_4o_mini,
    "azure/gpt-4o",
]

config = {
    "max_bootstrapped_demos": 4,
    "max_labeled_demos": 4,
    "num_candidate_programs": 5,
    "num_threads": 4,
}

for model_name in models:
    if "llama" in model_name:
        lm = dspy.LM(base_url=OLLAMA_URL, model=model_name)
    elif "azure/gpt-4o" == model_name:
        lm = dspy.LM(
            model_name,
            api_base=AZURE_OPENAI_ENDPOINT,
            api_version=AZURE_OPENAI_VERSION,
            api_key=AZURE_OPENAI_KEY,
            temperature=0,
            cache=True,
            max_tokens=2_000,
        )
    else:
        lm = dspy.LM(
            model_name,
            api_base=AZURE_OPENAI_ENDPOINT,
            api_version=AZURE_OPENAI_VERSION,
            # api_key=AZURE_OPENAI_KEY,
            temperature=0,
            cache=True,
            max_tokens=2_000,
        )
    dspy.settings.configure(lm=lm)
    model_name = model_name.replace("azure/", "")
    # no cooklang
    teleprompter = MIPROv2(
        metric=validate_answer,
        auto="light",  # Can choose between light, medium, and heavy optimization runs
        num_threads=10,
    )

    compiled_prompt_opt = teleprompter.compile(
        CookLangFormatterNoSteps(),
        trainset=examples,
        max_bootstrapped_demos=3,
        max_labeled_demos=5,
        requires_permission_to_run=False,
    )
    compiled_prompt_opt.save(
        f"data/programs/MIPROv2_{model_name}_without_cooklang.json"
    )

    optimizer = BootstrapFewShotWithRandomSearch(metric=validate_answer, **config)
    optimized_program = optimizer.compile(CookLangFormatterNoSteps(), trainset=examples)
    optimized_program.save(
        f"data/programs/RandomFewShot_{model_name}_without_cooklang.json"
    )

    CookLangFormatterNoSteps().save(
        f"data/programs/zeroshot_{model_name}_without_cooklang.json"
    )

    # no cooklang, no ingredients
    teleprompter = MIPROv2(
        metric=validate_answer,
        auto="light",  # Can choose between light, medium, and heavy optimization runs
        num_threads=10,
    )
    # CookLangFormatterNoStepsNoIngredients
    compiled_prompt_opt = teleprompter.compile(
        CookLangFormatterNoStepsNoIngredients(),
        trainset=examples_no_ingredients,
        max_bootstrapped_demos=3,
        max_labeled_demos=5,
        requires_permission_to_run=False,
    )
    compiled_prompt_opt.save(
        f"data/programs/MIPROv2_{model_name}_without_cooklang_spec_without_ings.json"
    )

    optimizer = BootstrapFewShotWithRandomSearch(metric=validate_answer, **config)
    optimized_program = optimizer.compile(
        CookLangFormatterNoStepsNoIngredients(), trainset=examples_no_ingredients
    )
    optimized_program.save(
        f"data/programs/RandomFewShot_{model_name}_without_cooklang_spec_without_ings.json"
    )

    CookLangFormatterNoStepsNoIngredients().save(
        f"data/programs/zeroshot_{model_name}_without_cooklang_spec_without_ings.json"
    )

    # with cooklang

    teleprompter = MIPROv2(
        metric=validate_answer,
        auto="light",  # Can choose between light, medium, and heavy optimization runs
        num_threads=10,
    )
    compiled_prompt_opt = teleprompter.compile(
        CookLangFormatter(),
        trainset=examples,
        max_bootstrapped_demos=3,
        max_labeled_demos=5,
        requires_permission_to_run=False,
    )
    compiled_prompt_opt.save(
        f"data/programs/MIPROv2_{model_name}_with_cooklang_spec.json"
    )

    optimizer = BootstrapFewShotWithRandomSearch(metric=validate_answer, **config)
    optimized_program = optimizer.compile(CookLangFormatter(), trainset=examples)
    optimized_program.save(
        f"data/programs/RandomFewShot_{model_name}_with_cooklang_spec.json"
    )

    CookLangFormatter().save(
        f"data/programs/zeroshot_{model_name}_with_cooklang_spec.json"
    )